In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：使用TensorFlow提交一个超参数调整训练任务

## 安装

将Google *云存储*库安装到您的设备上。

In [ ]:
! pip3 install google-cloud-storage

### 重启内核

安装了Vertex SDK和Google *cloud-storage*后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始前

### GPU 运行时

*如果您有此选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 GCP 项目

**无论您的笔记本环境如何，都需要执行以下步骤。**

1. [选择或创建 GCP 项目](https://console.cloud.google.com/cloud-resource-manager)。您首次创建账户时，将获得 $300 的免费信用额度，可用于计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud 笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 对以 `!` 为前缀的行作为 shell 命令执行，并将以 `$` 为前缀的 Python 变量插入到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改`REGION`变量，该变量用于笔记本的其余部分操作。以下是Vertex支持的地区。我们建议尽可能选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能使用多区域存储桶来训练Vertex。并非所有地区都支持所有Vertex服务。有关每个地区的最新支持，请参阅[Vertex服务的地区支持]()。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加实时教程会议，您可能会使用共享的测试账户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的GCP账户

**如果您正在使用谷歌云笔记本**，您的环境已经通过身份验证。请跳过这一步。

*注意：如果您使用Vertex笔记本并运行该单元格，则该单元格会自动跳过执行身份验证步骤。*

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### 创建一个云存储桶

**无论您使用的是什么笔记本环境，都需要执行以下步骤。**

本教程旨在使用公共云存储桶中的训练数据以及本地云存储桶进行批量预测。您也可以使用自己存储在本地云存储桶中的训练数据。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才运行以下代码以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

最后，通过检查其内容来验证对云存储桶的访问权限：

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex SDK

将Vertex SDK导入到我们的Python环境中。

In [ ]:
import json
import os
import sys
import time
from datetime import datetime

from googleapiclient import discovery

顶点常量

为顶点设置以下常量：

- `PARENT`：数据集、模型和端点资源的顶点位置根路径。

In [ ]:
# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## 客户端

Vertex SDK 作为客户端/服务器模型工作。在您的端（Python 脚本）中，您将创建一个客户端，该客户端向服务器（Vertex）发送请求并接收响应。

在本教程中，您将使用多个客户端，所以请提前设置好所有客户端。

In [ ]:
client = discovery.build("ml", "v1")

准备一份培训师指引稿。

### 包装组装

In [ ]:
# Make folder for python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\
# Requires TensorFlow Datasets\n\
setuptools.setup(\n\
    install_requires=[\n\
        'tensorflow_datasets==1.3.0',\n\
    ],\n\
    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Hyperparameter Tuning - Boston Housing\n\
Version: 0.0.0\n\
Summary: Demonstration hyperparameter tuning script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@gmail.com\n\
License: Public\n\
Description: Demo\n\
Platform: Vertex AI"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Task.py 的內容

In [ ]:
%%writefile custom/trainer/task.py
# Custom Training for Boston Housing
  
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default='/tmp/saved_model', type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--units', dest='units',
                    default=64, type=int,
                    help='Number of units.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

def make_dataset():
  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)
    
  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)

# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(args.units, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(args.units, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

model = build_and_compile_dnn_model()

# Instantiate the HyperTune reporting object
hpt = HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_loss',
        metric_value=logs['val_loss'],
        global_step=epoch)

# Train the model
BATCH_SIZE = 16
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=BATCH_SIZE, validation_split=0.1, callbacks=[HPTCallback()])
model.save(args.model_dir)


把培训脚本存储在你的云存储桶里

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz gs://$BUCKET_NAME/hpt_boston_housing.tar.gz

训练一个模型

[projects.jobs.create](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs/create)
[项目.就业.创造](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs/create)

#### 请求

In [ ]:
JOB_NAME = "hyperparameter_tuning_" + TIMESTAMP

training_input = {
    "scaleTier": "CUSTOM",
    "masterType": "n1-standard-4",
    "packageUris": ["gs://" + BUCKET_NAME + "/hpt_boston_housing.tar.gz"],
    "pythonModule": "trainer.task",
    "args": [
        "--model-dir=" + "gs://{}/{}".format(BUCKET_NAME, JOB_NAME),
    ],
    "region": REGION,
    "runtimeVersion": "2.1",
    "pythonVersion": "3.7",
}

# Add hyperparameter tuning to the job config.
hyperparams = {
    "goal": "MINIMIZE",
    "hyperparameterMetricTag": "val_loss",
    "maxTrials": 6,
    "maxParallelTrials": 1,
    "params": [],
    "algorithm": "RANDOM_SEARCH",
}

hyperparams["params"].append(
    {
        "parameterName": "lr",
        "type": "DISCRETE",
        "discreteValues": [0.001, 0.01, 0.1],
        "scaleType": "UNIT_LINEAR_SCALE",
    }
)

hyperparams["params"].append(
    {
        "parameterName": "units",
        "type": "INTEGER",
        "minValue": 32,
        "maxValue": 256,
        "scaleType": "UNIT_LINEAR_SCALE",
    }
)

# Add hyperparameter specification to the training inputs dictionary.
training_input["hyperparameters"] = hyperparams

body = {"jobId": JOB_NAME, "trainingInput": training_input}

request = (
    client.projects()
    .jobs()
    .create(
        parent="projects/" + PROJECT_ID,
    )
)
request.body = body

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().jobs().create(parent="projects/" + PROJECT_ID, body=body)

*示例输出*：
```
{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/jobs?alt=json",
  "method": "POST",
  "body": {
    "jobId": "hyperparameter_tuning_20210226020553",
    "trainingInput": {
      "scaleTier": "CUSTOM",
      "masterType": "n1-standard-4",
      "packageUris": [
        "gs://migration-ucaip-trainingaip-20210226020553/hpt_boston_housing.tar.gz"
      ],
      "pythonModule": "trainer.task",
      "args": [
        "--model-dir=gs://migration-ucaip-trainingaip-20210226020553/hyperparameter_tuning_20210226020553"
      ],
      "region": "us-central1",
      "runtimeVersion": "2.1",
      "pythonVersion": "3.7",
      "hyperparameters": {
        "goal": "MINIMIZE",
        "hyperparameterMetricTag": "val_loss",
        "maxTrials": 6,
        "maxParallelTrials": 1,
        "params": [
          {
            "parameterName": "lr",
            "type": "DISCRETE",
            "discreteValues": [
              0.001,
              0.01,
              0.1
            ],
            "scaleType": "UNIT_LINEAR_SCALE"
          },
          {
            "parameterName": "units",
            "type": "INTEGER",
            "minValue": 32,
            "maxValue": 256,
            "scaleType": "UNIT_LINEAR_SCALE"
          }
        ],
        "algorithm": "RANDOM_SEARCH"
      }
    }
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.jobs.create",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}
```

#### 电话

In [ ]:
result = request.execute()

#### 回应

In [ ]:
print(json.dumps(result, indent=2))

示例输出：
```
{
  "jobId": "hyperparameter_tuning_20210226020553",
  "trainingInput": {
    "scaleTier": "CUSTOM",
    "masterType": "n1-standard-4",
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210226020553/hpt_boston_housing.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210226020553/hyperparameter_tuning_20210226020553"
    ],
    "hyperparameters": {
      "goal": "MINIMIZE",
      "params": [
        {
          "parameterName": "lr",
          "type": "DISCRETE",
          "discreteValues": [
            0.001,
            0.01,
            0.1
          ],
          "scaleType": "UNIT_LINEAR_SCALE"
        },
        {
          "parameterName": "units",
          "minValue": 32,
          "maxValue": 256,
          "type": "INTEGER",
          "scaleType": "UNIT_LINEAR_SCALE"
        }
      ],
      "maxTrials": 6,
      "maxParallelTrials": 1,
      "hyperparameterMetricTag": "val_loss",
      "algorithm": "RANDOM_SEARCH"
    },
    "region": "us-central1",
    "runtimeVersion": "2.1",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-02-26T02:06:01Z",
  "state": "QUEUED",
  "trainingOutput": {
    "isHyperparameterTuningJob": true
  },
  "etag": "MPeWDTbQOUQ="
}
```

In [ ]:
# The short numeric ID for the custom training job
hyperparameter_tuning_short_id = result["jobId"]
# The full unique ID for the custom training job
hyperparameter_tuning_id = "projects/" + PROJECT_ID + "/jobs/" + result["jobId"]

print(hyperparameter_tuning_id)

### [projects.jobs.get](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs/get)

### [项目.作业.获取](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs/get)

#### 电话

In [ ]:
request = client.projects().jobs().get(name=hyperparameter_tuning_id)

result = request.execute()

#### 回复

In [ ]:
print(json.dumps(result, indent=2))

*示例输出*：
```
{
  "jobId": "hyperparameter_tuning_20210226020553",
  "trainingInput": {
    "scaleTier": "CUSTOM",
    "masterType": "n1-standard-4",
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210226020553/hpt_boston_housing.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210226020553/hyperparameter_tuning_20210226020553"
    ],
    "hyperparameters": {
      "goal": "MINIMIZE",
      "params": [
        {
          "parameterName": "lr",
          "type": "DISCRETE",
          "discreteValues": [
            0.001,
            0.01,
            0.1
          ],
          "scaleType": "UNIT_LINEAR_SCALE"
        },
        {
          "parameterName": "units",
          "minValue": 32,
          "maxValue": 256,
          "type": "INTEGER",
          "scaleType": "UNIT_LINEAR_SCALE"
        }
      ],
      "maxTrials": 6,
      "maxParallelTrials": 1,
      "hyperparameterMetricTag": "val_loss",
      "algorithm": "RANDOM_SEARCH"
    },
    "region": "us-central1",
    "runtimeVersion": "2.1",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-02-26T02:06:01Z",
  "state": "QUEUED",
  "trainingOutput": {
    "isHyperparameterTuningJob": true
  },
  "etag": "GNEDjq+ds8I="
}
```

## 等待研究完成

In [ ]:
while True:
    response = client.projects().jobs().get(name=hyperparameter_tuning_id).execute()

    if response["state"] != "SUCCEEDED":
        print("Study trials have not completed:", response["state"])
        if response["state"] == "FAILED":
            break
    else:
        print("Study trials have completed:")
        break
    time.sleep(60)

In [ ]:
print(json.dumps(response, indent=2))

{
  "jobId": "hyperparameter_tuning_20210226020553",
  "trainingInput": {
    "scaleTier": "CUSTOM",
    "masterType": "n1-standard-4",
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210226020553/hpt_boston_housing.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210226020553/hyperparameter_tuning_20210226020553"
    ],
    "hyperparameters": {
      "goal": "MINIMIZE",
      "params": [
        {
          "parameterName": "lr",
          "type": "DISCRETE",
          "discreteValues": [
            0.001,
            0.01,
            0.1
          ],
          "scaleType": "UNIT_LINEAR_SCALE"
        },
        {
          "parameterName": "units",
          "minValue": 32,
          "maxValue": 256,
          "type": "INTEGER",
          "scaleType": "UNIT_LINEAR_SCALE"
        }
      ],
      "maxTrials": 6,
      "maxParallelTrials": 1,
      "hyperparameterMetricTag": "val_loss",
      "algorithm": "RANDOM_SEARCH"
    },
    "region": "us-central1",
    "runtimeVersion": "2.1",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-02-26T02:06:01Z",
  "startTime": "2021-02-26T02:06:03Z",
  "endTime": "2021-02-26T02:35:25Z",
  "state": "SUCCEEDED",
  "trainingOutput": {
    "completedTrialCount": "6",
    "trials": [
      {
        "trialId": "3",
        "hyperparameters": {
          "lr": "0.1",
          "units": "39"
        },
        "finalMetric": {
          "trainingStep": "8",
          "objectiveValue": 45.76672372585389
        },
        "startTime": "2021-02-26T02:16:50.943369954Z",
        "endTime": "2021-02-26T02:20:15Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "5",
        "hyperparameters": {
          "lr": "0.1",
          "units": "169"
        },
        "finalMetric": {
          "trainingStep": "19",
          "objectiveValue": 46.09460951642292
        },
        "startTime": "2021-02-26T02:26:23.876101420Z",
        "endTime": "2021-02-26T02:29:59Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "2",
        "hyperparameters": {
          "lr": "0.1",
          "units": "69"
        },
        "finalMetric": {
          "trainingStep": "19",
          "objectiveValue": 51.7642993461795
        },
        "startTime": "2021-02-26T02:12:03.812631906Z",
        "endTime": "2021-02-26T02:15:30Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "1",
        "hyperparameters": {
          "lr": "0.01",
          "units": "232"
        },
        "finalMetric": {
          "trainingStep": "19",
          "objectiveValue": 55.58717541578339
        },
        "startTime": "2021-02-26T02:06:40.790962827Z",
        "endTime": "2021-02-26T02:10:47Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "6",
        "hyperparameters": {
          "lr": "0.001",
          "units": "123"
        },
        "finalMetric": {
          "trainingStep": "19",
          "objectiveValue": 76.20946111911681
        },
        "startTime": "2021-02-26T02:31:11.163541810Z",
        "endTime": "2021-02-26T02:34:36Z",
        "state": "SUCCEEDED"
      },
      {
        "trialId": "4",
        "hyperparameters": {
          "lr": "0.01",
          "units": "246"
        },
        "finalMetric": {
          "trainingStep": "11",
          "objectiveValue": 100.75884600383479
        },
        "startTime": "2021-02-26T02:21:36.635969861Z",
        "endTime": "2021-02-26T02:25:02Z",
        "state": "SUCCEEDED"
      }
    ],
    "consumedMLUnits": 0.39,
    "isHyperparameterTuningJob": true,
    "hyperparameterMetricTag": "val_loss"
  },
  "etag": "eUgnylIe/+0="
}
```

请审查研究的结果.

In [ ]:
best = (None, None, None, 0.0)
response = client.projects().jobs().get(name=hyperparameter_tuning_id).execute()
for trial in response["trainingOutput"]["trials"]:
    print(json.dumps(trial, indent=2))
    # Keep track of the best outcome
    try:
        if float(trial.final_measurement.metrics[0].value) > best[3]:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
    except:
        pass

print()
print("ID", best[0])
print("Decay", best[1])
print("Learning Rate", best[2])
print("Validation Accuracy", best[3])

{
  "trialId": "3",
  "hyperparameters": {
    "lr": "0.1",
    "units": "39"
  },
  "finalMetric": {
    "trainingStep": "8",
    "objectiveValue": 45.76672372585389
  },
  "startTime": "2021-02-26T02:16:50.943369954Z",
  "endTime": "2021-02-26T02:20:15Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "5",
  "hyperparameters": {
    "lr": "0.1",
    "units": "169"
  },
  "finalMetric": {
    "trainingStep": "19",
    "objectiveValue": 46.09460951642292
  },
  "startTime": "2021-02-26T02:26:23.876101420Z",
  "endTime": "2021-02-26T02:29:59Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "2",
  "hyperparameters": {
    "lr": "0.1",
    "units": "69"
  },
  "finalMetric": {
    "trainingStep": "19",
    "objectiveValue": 51.7642993461795
  },
  "startTime": "2021-02-26T02:12:03.812631906Z",
  "endTime": "2021-02-26T02:15:30Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "1",
  "hyperparameters": {
    "lr": "0.01",
    "units": "232"
  },
  "finalMetric": {
    "trainingStep": "19",
    "objectiveValue": 55.58717541578339
  },
  "startTime": "2021-02-26T02:06:40.790962827Z",
  "endTime": "2021-02-26T02:10:47Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "6",
  "hyperparameters": {
    "lr": "0.001",
    "units": "123"
  },
  "finalMetric": {
    "trainingStep": "19",
    "objectiveValue": 76.20946111911681
  },
  "startTime": "2021-02-26T02:31:11.163541810Z",
  "endTime": "2021-02-26T02:34:36Z",
  "state": "SUCCEEDED"
}
{
  "trialId": "4",
  "hyperparameters": {
    "lr": "0.01",
    "units": "246"
  },
  "finalMetric": {
    "trainingStep": "11",
    "objectiveValue": 100.75884600383479
  },
  "startTime": "2021-02-26T02:21:36.635969861Z",
  "endTime": "2021-02-26T02:25:02Z",
  "state": "SUCCEEDED"
}

ID None
Decay None
Learning Rate None
Validation Accuracy 0.0
}

清理工作

要清理此项目中使用的所有GCP资源，您可以[删除用于本教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = True

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME